# Bayesian Dynamic Linear Modelling with Multiple Processing

Bayesian dynamic linear model is a promising method for time series data analysis and short-term forecasting. I used the model a few year ago in an attempt to sole a kaggle competition for the M5 Forecasting competition. However due to the limitations in time the model could only forecast 50% of the data before the cluster expired. A lot could be done to optimize the run like removing backawards and forward components of the forecating. this however sacrifices the accuracy and maximizes the runtime. This notebook utilizes multiple processing to parallelize the workload across multiple model keys.

### install pydlm

In [ ]:
#pip install pydlm

In [ ]:
#

In [ ]:
#import packages
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from pydlm import dlm, trend, seasonality
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import boxcox
from scipy.special import inv_boxcox
import os
import gc
import time
import tqdm
import datetime

In [ ]:
#the function reduces dataframe size to keep the notebook from breaking when memory is full
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [ ]:
#Import and transform data
sales_train_validation = reduce_mem_usage(pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv'))
calendar = reduce_mem_usage(pd.read_csv('/kaggle/input/m5-forecasting-accuracy/calendar.csv'))
sell_prices = reduce_mem_usage(pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sell_prices.csv'))
submission_sample = reduce_mem_usage(pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sample_submission.csv'))


sell_prices['id'] = sell_prices.item_id+'_'+ sell_prices.store_id+'_validation'
ex_columns = ['item_id','dept_id','cat_id','store_id','state_id']
sales_train_validation = reduce_mem_usage(sales_train_validation.drop(ex_columns, axis = 1))
sales_train = reduce_mem_usage(sales_train_validation.melt(id_vars=["id"], 
        var_name="d", 
        value_name="sales_units"))


day_d = reduce_mem_usage(calendar[['date','d','wm_yr_wk']])
sales_date = reduce_mem_usage(sales_train.merge(day_d, on = 'd', how = 'left'))
sell_prices = reduce_mem_usage(sell_prices[['id','sell_price','wm_yr_wk']])

df_final = reduce_mem_usage(sales_date.merge(sell_prices, on=['id','wm_yr_wk'], how = 'left'))
df_final['y'] = df_final['sales_units']
x_trans, lamb = boxcox(df_final['y']+1)
df_final['y'] = x_trans

#create holidays data frame

event_name = calendar[['event_name_1','date']].dropna(axis = 0)
event_name.columns = ['holiday','ds']
event_name['lower_window'] = 0
event_name['upper_window'] = 1
#reduce dataframe size
#Limit the data to at least 1 years of data
min_date = pd.to_datetime(df_final['date'].max()) - datetime.timedelta(366)
historic_data = df_final[df_final['date'] >= min_date.strftime("%Y-%m-%d")]
historic_data = reduce_mem_usage(historic_data)
event_name = reduce_mem_usage(event_name)
submission_columns = submission_sample.columns
lists_ids = historic_data['id'].unique()
#remove unwanted dfs
sales_date=pd.DataFrame()
calendar=pd.DataFrame()
sell_prices=pd.DataFrame()
sales_train_validation=pd.DataFrame()
sales_train=pd.DataFrame()
day_d=pd.DataFrame()
sales_date=pd.DataFrame()
df_final=pd.DataFrame()
gc.collect()

In [ ]:
#forcasting functions
def forecast_df(lists_ids):
    df = historic_data.loc[historic_data['id']==lists_ids]
    return df

def baysian_forecast(lists_ids):
    hist = forecast_df(lists_ids)
    hist_data = hist['y'].values
    linear_trend = trend(degree=1, discount=0.95, name='linear_trend', w=100)
    # weekly seasonality
    seasonal52 = seasonality(period=52, discount=0.99, name='seasonal52', w=1.0)
    # Build a simple dlm
    simple_dlm = dlm(hist_data) + linear_trend + seasonal52 
    # Fit the model
    simple_dlm.fit()
    forecast = simple_dlm.predictN(date=(len(hist_data) - 1), N=28)[0]
    forecast = (inv_boxcox(forecast, lamb).round()-1)
    forecast = forecast.replace(np.inf, np.nan).replace(-np.inf, np.nan).replace(-0, 0).replace(np.nan, 0)
    forecast = forecast.fillna(0)
    forecast_array = np.append(np.array([lists_ids]),forecast)
    return forecast_array

In [ ]:
#use multiple processing in code
from multiprocessing import Pool, cpu_count
print(f'Parallelism on {cpu_count()} CPU')

start_time = time.time()
forecast_array = []
with Pool(10) as p:
    predictions = list(p.imap_unordered(baysian_forecast, lists_ids))
    
submission_df = pd.DataFrame(predictions)
submission_df.columns = submission_columns
submission_df.to_csv('submission.csv', header='column_names', index=False)

print("--- %s seconds ---" % (time.time() - start_time))